<a href="https://colab.research.google.com/github/ZeeTsing/Carvana_challenge/blob/master/3_Unet_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up notebook

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import PIL
from tensorflow.keras.layers import Dense, Conv2D, Input, MaxPool2D, UpSampling2D, Concatenate, Conv2DTranspose
from tensorflow.keras import Sequential,Model
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
import os
from tensorflow.keras.backend import flatten
import tensorflow.keras.backend as K


# Prepare data generator (no augmentation)

In [0]:
data_dir = '/content/drive/My Drive/car_data/train/'
mask_dir = '/content/drive/My Drive/car_data/train_masks/'

all_images = os.listdir(data_dir)

to_train = 1 # ratio of number of train set images to use
total_train_images = all_images[:int(len(all_images)*to_train)]

WIDTH = 512  #  actual : 1918//1920 divisive by 64
HEIGHT = 512 # actual : 1280
BATCH_SIZE = 5

In [0]:
# split train set and test set
train_images, validation_images = train_test_split(total_train_images, train_size=0.8, test_size=0.2,random_state = 0)

In [0]:
# generator that we will use to read the data from the directory
def data_gen_small(data_dir, mask_dir, images, batch_size, dims):
        """
        data_dir: where the actual images are kept
        mask_dir: where the actual masks are kept
        images: the filenames of the images we want to generate batches from
        batch_size: self explanatory
        dims: the dimensions in which we want to rescale our images, tuple
        """
        while True:
            ix = np.random.choice(np.arange(len(images)), batch_size)
            imgs = []
            labels = []
            for i in ix:
                # images
                original_img = load_img(data_dir + images[i])
                resized_img = original_img.resize(dims)
                array_img = img_to_array(resized_img)/255
                imgs.append(array_img)
                
                # masks
                original_mask = load_img(mask_dir + images[i].split(".")[0] + '_mask.gif')
                resized_mask = original_mask.resize(dims)
                array_mask = img_to_array(resized_mask)/255
                labels.append(array_mask[:, :, 0])

            imgs = np.array(imgs)
            labels = np.array(labels)
            yield imgs, labels.reshape(-1, dims[0], dims[1], 1)

In [0]:
# generator that we will use to read the data from the directory with random augmentation
def data_gen_aug(data_dir, mask_dir, images, batch_size, dims):
        """
        data_dir: where the actual images are kept
        mask_dir: where the actual masks are kept
        images: the filenames of the images we want to generate batches from
        batch_size: self explanatory
        dims: the dimensions in which we want to rescale our images, tuple
        """
        while True:
            ix = np.random.choice(np.arange(len(images)), batch_size)
            imgs = []
            labels = []
            for i in ix:
                # read images and masks
                original_img = load_img(data_dir + images[i])
                original_mask = load_img(mask_dir + images[i].split(".")[0] + '_mask.gif')
                
                # transform into ideal sizes
                resized_img = original_img.resize(dims)
                resized_mask = original_mask.resize(dims)
              
                # add random augmentation > here we only flip horizontally
                if np.random.random() < 0.5:
                  resized_img = resized_img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                  resized_mask = resized_mask.transpose(PIL.Image.FLIP_LEFT_RIGHT)

                array_img = img_to_array(resized_img)/255
                array_mask = img_to_array(resized_mask)/255

                imgs.append(array_img)
                labels.append(array_mask[:, :, 0])
                
            imgs = np.array(imgs)
            labels = np.array(labels)
            yield imgs, labels.reshape(-1, dims[0], dims[1], 1)

In [0]:
#generator for train and validation data set
train_gen = data_gen_aug(data_dir, mask_dir, train_images, BATCH_SIZE, (WIDTH, HEIGHT))
val_gen = data_gen_small(data_dir, mask_dir, validation_images, BATCH_SIZE, (WIDTH, HEIGHT))

# Set up Unet model

Define down and up layers that will be used in Unet model

In [0]:
def down(input_layer, filters, pool=True):
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(input_layer)
    residual = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    if pool:
        max_pool = MaxPool2D()(residual)
        return max_pool, residual
    else:
        return residual

def up(input_layer, residual, filters):
    filters=int(filters)
    upsample = UpSampling2D()(input_layer)
    upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
    concat = Concatenate(axis=3)([residual, upconv])
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(concat)
    conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    return conv2  

In [0]:
# Make a custom U-nets implementation.
filters = 64
input_layer = Input(shape = [WIDTH, HEIGHT, 3])
layers = [input_layer]
residuals = []

# Down 1
d1, res1 = down(input_layer, filters)
residuals.append(res1)

filters *= 2

# Down 2
d2, res2 = down(d1, filters)
residuals.append(res2)

filters *= 2

# Down 3
d3, res3 = down(d2, filters)
residuals.append(res3)

filters *= 2

# Down 4
d4, res4 = down(d3, filters)
residuals.append(res4)

filters *= 2

# Down 5
d5 = down(d4, filters, pool=False)

# Up 1
up1 = up(d5, residual=residuals[-1], filters=filters/2)
filters /= 2

# Up 2
up2 = up(up1, residual=residuals[-2], filters=filters/2)

filters /= 2

# Up 3
up3 = up(up2, residual=residuals[-3], filters=filters/2)

filters /= 2

# Up 4
up4 = up(up3, residual=residuals[-4], filters=filters/2)

out = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(up4)

model = Model(input_layer, out)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [0]:
# Now let's use Tensorflow to write our own dice_coeficcient metric, which is a effective indicator of how much two sets overlap with each other
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = flatten(y_true)
    y_pred_f = flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


In [0]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "/content/drive/My Drive/car_data/unet_cp_full_512batch5/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8,
                                              restore_best_weights=False
                                              )

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.2,
                                   patience=3,
                                   verbose=1,
                                   min_delta=1e-3,min_lr = 1e-6
                                   )

adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Train model

Load weights and continue training

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/content/drive/My Drive/car_data/unet_cp_full_512batch5/cp-0016.ckpt'

In [0]:
model.load_weights(latest)

In [0]:
model.compile(optimizer = adam, loss = BinaryCrossentropy(), metrics=['accuracy',dice_coef])

In [0]:
model.fit(train_gen, callbacks=[cp_callback,early_stop,reduce_lr],
                    steps_per_epoch=np.ceil(float(len(train_images)) / float(BATCH_SIZE)),
                    epochs=100,
                    validation_steps=np.ceil(float(len(validation_images)) / float(BATCH_SIZE)),
                    validation_data = val_gen)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 814.0 steps, validate for 204.0 steps
Epoch 1/100
813/814 [============================>.] - ETA: 0s - loss: 0.0071 - accuracy: 0.9935 - dice_coef: 0.9903
Epoch 00001: saving model to /content/drive/My Drive/car_data/unet_cp_full_512batch5/cp-0001.ckpt
814/814 [==============================] - 681s 836ms/step - loss: 0.0071 - accuracy: 0.9935 - dice_coef: 0.9903 - val_loss: 0.0060 - val_accuracy: 0.9939 - val_dice_coef: 0.9919
Epoch 2/100
813/814 [============================>.] - ETA: 0s - loss: 0.0055 - accuracy: 0.9940 - dice_coef: 0.9922
Epoch 00002: saving model to /content/drive/My Drive/car_data/unet_cp_full_512batch5/cp-0002.ckpt
814/814 [==============================] - 681s 837ms/step - loss: 0.0055 - accuracy: 0.9940 - dice_coef: 0.9922 - val_loss: 0.0060 - val_accuracy: 0.9939 - val_dice_coef: 0.9920
Epoch 3/100
813/814 [============================>.] - ETA: 0s - loss: 0.0059 - accuracy: 0.9939 - dice_coef: 0.99